In [15]:
import os,sys,glob
import rasterio as rio
import geopandas as gpd
import pandas as pd
import numpy as np
import tensorflow as tf
from rasterio.plot import reshape_as_image
from tensorflow.keras.layers import BatchNormalization,Dropout
from datetime import datetime
from tensorflow.keras import models,layers,Input
from tensorflow.keras import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from shapely import wkt

training_path = "Input/sentinel/test_data_from_drive/patches_all/normalised_train/"
target_file_path = "Input/Target/concat/target_yield.shp"
model_path = "unet_multi_output/model_BSize_256_NEpochs_10_2023-10-30 15:48:34.412397.h5"
unet_model = models.load_model(model_path, compile=False)


In [2]:
mse = tf.keras.metrics.MeanSquaredError()
rmse = tf.keras.metrics.RootMeanSquaredError()
mae = tf.keras.metrics.MeanAbsoluteError()
# mape = tf.keras.metrics.MeanAbsolutePercentageError()
msle = tf.keras.metrics.MeanSquaredLogarithmicError()
# cos_sim = tf.keras.metrics.CosineSimilarity(axis=1)
# log_cos = tf.keras.metrics.LogCoshError()
config = {
"epochs":75,
"batch_size":256,
"loss_function":'mse',
# "metrics":[mse,rmse,mae,mape,msle,cos_sim,log_cos],
"metrics":[mse,mae,msle],
"learning_rate":0.0001
# "optimizer":'adam'
}
wandb.init(project="unet_multi_output", entity="msc-thesis",config=config)
now = datetime.now()
date_time = now.strftime("%d_%m_%Y_%H_%M_%S")

wandb.run.name = wandb.run.id+"_"+date_time

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aarun-vishwanath (msc-thesis). Use `wandb login --relogin` to force relogin


In [3]:
def read_training():
    training_file_list = glob.glob(os.path.join(training_path,"*.tif"))
    target_gdf = gpd.read_file(target_file_path)
    print("Total Number of Patches:",len(training_file_list))
    ignore_patch_list = list()
    x = list()
    y = list()
    X_train = list()
    X_test = list()
    y_train = list()
    y_test = list()
    count = 0 
    for file in training_file_list:

        patch_src = rio.open(file)
        f_name = file.split("/")[-1].split(".")[0]
        patch_src_read = reshape_as_image(patch_src.read()[0:12]) ## Change the index here to add or remove the mask layer
        # print(0)
#         if patch_src_read.shape != patch_dim:
#             ignore_patch_list.append(f_name)
#             # print("Patch Dimensions Mismatch, skipping patch : {}".format(f_name))
#             continue

#         # print(1)
#         if np.isnan(patch_src_read).any():
#             # print("Has Nan values, skipping patch : {}".format(f_name))
#             continue

        # print(2)
        query = target_gdf.query(f"patch_name == '{f_name}'")["ykg_by_e7"]
        if len(query) != 1:
            # print("patch has no target value, skipping patch : {}".format(f_name))
            continue
        # print(patch_src_read[:,:,0:12].shape)
        # print(patch_src_read[:,:,12].shape)

        x.append(patch_src_read)
        y.append(float(query))

        patch_src.close()
        # print(count)
        count +=1
        # if count > 100:
        #     break

    # self.y = self.scaler.fit_transform(np.array(self.y).reshape(-1, 1))
    y = np.array(y)
    y = np.expand_dims(y,-1)
    x = np.array(x)
    
    # x = (x-np.min(x))/(np.max(x)-np.min(x))
    print("Any Null values? ",np.isnan(x).any())
    # print(self.y)
    # self.x = np.nan_to_num(self.x, nan=0)# Check for different value for no data
    print(f"x shape :{x.shape}, y shape: {y.shape}")
    # print(np.nanmin(self.x),np.nanmax(self.x))
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
    return X_train, X_test, y_train, y_test
    #Also, split the training into train and val
    # For testing, keep a part of the dataset as seperate (final month)

X_train, X_test, y_train, y_test = read_training()

Total Number of Patches: 12411
Any Null values?  False
x shape :(12411, 256, 256, 12), y shape: (12411, 1)


In [4]:
# unet_model.summary()

def get_encoder(old_model: Model) -> Model:
    # Get encoder
    encoder_input: Model = Model(inputs=old_model.layers[0].input,
                          outputs=old_model.layers[17].output)

    # Create Global Average Pooling.
    encoder_output = GlobalAveragePooling2D()(encoder_input.layers[-1].output)

    # Create the encoder adding the GAP layer as output.
    encoder: Model = Model(encoder_input.input, encoder_output, name='encoder')

    return encoder_input

In [5]:
unet_encoder = get_encoder(unet_model)

In [6]:
unet_encoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 12)]    0         
                                                                 
 conv2d_38 (Conv2D)          (None, 256, 256, 64)      6976      
                                                                 
 conv2d_39 (Conv2D)          (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 dropout_16 (Dropout)        (None, 128, 128, 64)      0         
                                                                 
 conv2d_40 (Conv2D)          (None, 128, 128, 128)     73856     
                                                             

In [33]:
unet_encoder.trainable = False

In [34]:
print("weights:",len(unet_encoder.weights))
print("trainable weights:",len(unet_encoder.trainable_weights))
print("Non trainable weights:",len(unet_encoder.non_trainable_weights))

weights: 18
trainable weights: 0
Non trainable weights: 18


In [35]:
# Get a copy of the weights of the Unet encoder
unet_encoder_weights = unet_encoder.get_weights()

In [36]:
model = models.Sequential()
# inputs = Input(shape=(256,256,12))

model.add(unet_encoder)
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(32, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(BatchNormalization())

model.add(layers.Dense(64, activation='relu')) # Add another dense layer
model.add(Dropout(0.5))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(1,activation='linear'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config["learning_rate"]),
              loss=config["loss_function"], metrics=config["metrics"])

In [37]:
len(model.trainable_weights)

14

In [38]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 16, 16, 1024)      9410176   
                                                                 
 flatten_3 (Flatten)         (None, 262144)            0         
                                                                 
 batch_normalization_3 (Batc  (None, 262144)           1048576   
 hNormalization)                                                 
                                                                 
 dense_18 (Dense)            (None, 64)                16777280  
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_19 (Dense)            (None, 32)                2080      
                                                      

In [39]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
history = model.fit(X_train, y_train,
            validation_data = (X_test, y_test),
            callbacks=[WandbCallback(),es],
            # callbacks=[es],
            epochs=config["epochs"],
            batch_size=config["batch_size"],
            verbose=1)

Epoch 1/75


2023-11-01 21:43:28.752179: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 29280436224 exceeds 10% of free system memory.


37/37 [==============================] - ETA: 0s - loss: 1.6846 - mean_squared_error: 1.8717 - mean_absolute_error: 1.0310 - mean_squared_logarithmic_error: 0.2725 

INFO:tensorflow:Assets written to: /home/jovyan/MSC_Thesis/MSc_Thesis_2023/wandb/run-20231101_212949-tpd4yg53/files/model-best/assets


INFO:tensorflow:Assets written to: /home/jovyan/MSC_Thesis/MSc_Thesis_2023/wandb/run-20231101_212949-tpd4yg53/files/model-best/assets
wandb: Adding directory to artifact (/home/jovyan/MSC_Thesis/MSc_Thesis_2023/wandb/run-20231101_212949-tpd4yg53/files/model-best)... Done. 4.9s


...

INFO:tensorflow:Assets written to: /home/jovyan/MSC_Thesis/MSc_Thesis_2023/wandb/run-20231101_212949-tpd4yg53/files/model-best/assets
wandb: Adding directory to artifact (/home/jovyan/MSC_Thesis/MSc_Thesis_2023/wandb/run-20231101_212949-tpd4yg53/files/model-best)... Done. 5.1s


37/37 [==============================] - 579s 16s/step - loss: 0.4831 - mean_squared_error: 0.4831 - mean_absolute_error: 0.5164 - mean_squared_logarithmic_error: 0.0754 - val_loss: 0.8744 - val_mean_squared_error: 0.8744 - val_mean_absolute_error: 0.7264 - val_mean_squared_logarithmic_error: 0.1164
Epoch 64/75
37/37 [==============================] - 564s 15s/step - loss: 0.4756 - mean_squared_error: 0.4756 - mean_absolute_error: 0.5169 - mean_squared_logarithmic_error: 0.0752 - val_loss: 1.0459 - val_mean_squared_error: 1.0459 - val_mean_absolute_error: 0.8074 - val_mean_squared_logarithmic_error: 0.1344
Epoch 65/75
37/37 [==============================] - 563s 15s/step - loss: 0.4819 - mean_squared_error: 0.4819 - mean_absolute_error: 0.5199 - mean_squared_logarithmic_error: 0.0746 - val_loss: 1.1338 - val_mean_squared_error: 1.1338 - val_mean_absolute_error: 0.8448 - val_mean_squared_logarithmic_error: 0.1415
Epoch 66/75
37/37 [==============================] - 564s 15s/step - loss

INFO:tensorflow:Assets written to: /home/jovyan/MSC_Thesis/MSc_Thesis_2023/wandb/run-20231101_212949-tpd4yg53/files/model-best/assets


INFO:tensorflow:Assets written to: /home/jovyan/MSC_Thesis/MSc_Thesis_2023/wandb/run-20231101_212949-tpd4yg53/files/model-best/assets
wandb: Adding directory to artifact (/home/jovyan/MSC_Thesis/MSc_Thesis_2023/wandb/run-20231101_212949-tpd4yg53/files/model-best)... Done. 5.0s


37/37 [==============================] - 578s 16s/step - loss: 0.4796 - mean_squared_error: 0.4796 - mean_absolute_error: 0.5212 - mean_squared_logarithmic_error: 0.0742 - val_loss: 0.7862 - val_mean_squared_error: 0.7862 - val_mean_absolute_error: 0.6844 - val_mean_squared_logarithmic_error: 0.1075
Epoch 71/75
37/37 [==============================] - 564s 15s/step - loss: 0.4629 - mean_squared_error: 0.4629 - mean_absolute_error: 0.5128 - mean_squared_logarithmic_error: 0.0735 - val_loss: 0.8998 - val_mean_squared_error: 0.8998 - val_mean_absolute_error: 0.7397 - val_mean_squared_logarithmic_error: 0.1198
Epoch 72/75
37/37 [==============================] - 566s 15s/step - loss: 0.4739 - mean_squared_error: 0.4739 - mean_absolute_error: 0.5151 - mean_squared_logarithmic_error: 0.0734 - val_loss: 0.8783 - val_mean_squared_error: 0.8783 - val_mean_absolute_error: 0.7301 - val_mean_squared_logarithmic_error: 0.1178
Epoch 73/75
37/37 [==============================] - 565s 15s/step - loss

In [19]:
def run_prediction(patch_src_read,f_name):
    # print(patch_src_read.shape)
    img_array = image.img_to_array(patch_src_read[:,:,0:12])
    img_batch = np.expand_dims(img_array, axis=0)
    prediction = unet_encoder.predict(img_batch)
    print(prediction.shape)
    print(prediction.min())
    print(prediction.max())

    # plt.imshow(prediction[0][:,:,0])
    # plt.imsave(os.path.join(self.output_eval_dir,f_name+".png"),prediction[0][:,:,0],cmap=cm.gray)
    # self.pred_val.append(prediction[0])
    # self.test_patches.append(img_batch)
    return prediction

In [20]:
target_file_path = "Input/Target/concat/target_yield.shp"
patch_name_list = list()
patch_geom_list = list()
output_eval_dir = "Output/Evaluation/unet_segmented/model_BSize_256_NEpochs_10_2023-10-30 15:48:34.412397"
def read_test():
    test_file_list = glob.glob(os.path.join(test_path,"*.tif"))
    target_gdf = gpd.read_file(target_file_path)
    print("Total Number of Patches:",len(test_file_list))
    pred_df = dict()
    count = 0 

    for file in test_file_list:

        patch_src = rio.open(file)
        f_name = file.split("/")[-1].split(".")[0]
        patch_src_read = reshape_as_image(patch_src.read()) #Change if want to include mask layer
        rec= target_gdf.query(f"patch_name == '{f_name}'")
        query = rec["ykg_by_e7"]
        if len(query) != 1:
            # print("patch has no target value, skipping patch : {}".format(f_name))
            continue
        # self.test_patches.append(patch_src_read)  

        # patch_name_list.append(f_name)
        # patch_geom_list.append(rec["geometry"].iloc[0])

        prediction = run_prediction(patch_src_read,f_name)
        # meta = patch_src.meta.copy()
        # meta.update(
        #     dtype=rio.float32,
        #     count=1)
        # outpath = output_eval_dir+"/"+f_name+".tif"
        # with rio.open(outpath, 'w', **meta) as outds:
        #     outds.write_band(1,prediction)
        # self.true_val.append(float(query))
        patch_src.close()
        return prediction
        count+=1
        if count == 1:
            break




In [21]:
pred = read_test()

Total Number of Patches: 2013
1/1 [==============================] - 0s 25ms/step
(1, 16, 16, 1024)
0.0
1.6201233


In [22]:
pred.shape

(1, 16, 16, 1024)